In [1]:
#pip install tensorflow

In [1]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
directory = "TImages/"

In [3]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [4]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [5]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [6]:
create_training_data()

In [7]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [8]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [11]:
## NB MODEL

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [14]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [38]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5] # You can adjust this range
    
}

In [39]:
# Create the Gaussian Naive Bayes model
nb = GaussianNB()

In [40]:
# Create the GridSearch object without cross-validation
grid_search = GridSearchCV(nb, param_grid, cv=5, verbose=2)  # Set cv=None for no cross-validation

In [41]:
# priors
grid_search.class_prior_ = [0.5, 0.5]

In [42]:
# Fit the GridSearch object to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END ................................var_smoothing=1e-09; total time=   8.5s
[CV] END ................................var_smoothing=1e-09; total time=   8.4s
[CV] END ................................var_smoothing=1e-09; total time=   8.4s
[CV] END ................................var_smoothing=1e-09; total time=   8.4s
[CV] END ................................var_smoothing=1e-09; total time=   8.4s
[CV] END ................................var_smoothing=1e-08; total time=   8.5s
[CV] END ................................var_smoothing=1e-08; total time=   8.4s
[CV] END ................................var_smoothing=1e-08; total time=   8.5s
[CV] END ................................var_smoothing=1e-08; total time=   8.5s
[CV] END ................................var_smoothing=1e-08; total time=   8.5s
[CV] END ................................var_smoothing=1e-07; total time=   8.5s
[CV] END ................................var_smoo

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             verbose=2)

In [43]:
# Get the best estimator and evaluate it
# best_nb = grid_search.best_estimator_
y_test_pred = best_nb.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
# print("Best Gaussian Naive Bayes Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 35.90%


In [ ]:
# work progressing...........


In [44]:
grid_search.cv_results_

{'mean_fit_time': array([4.79142475, 4.77848492, 4.78803177, 4.79120655, 4.77843685]),
 'std_fit_time': array([0.06491218, 0.02294985, 0.01438451, 0.02869476, 0.006868  ]),
 'mean_score_time': array([3.76090374, 3.81684489, 3.82170448, 3.8268291 , 3.82548099]),
 'std_score_time': array([0.03010367, 0.01987339, 0.00635089, 0.01349803, 0.00735333]),
 'param_var_smoothing': masked_array(data=[1e-09, 1e-08, 1e-07, 1e-06, 1e-05],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'var_smoothing': 1e-09},
  {'var_smoothing': 1e-08},
  {'var_smoothing': 1e-07},
  {'var_smoothing': 1e-06},
  {'var_smoothing': 1e-05}],
 'split0_test_score': array([0.43589744, 0.35576923, 0.13034188, 0.05662393, 0.02884615]),
 'split1_test_score': array([0.30128205, 0.19337607, 0.11752137, 0.06089744, 0.03311966]),
 'split2_test_score': array([0.27029915, 0.17521368, 0.10683761, 0.04807692, 0.02777778]),
 'split3_test_score': array([0.38247863

In [45]:
df = pd.DataFrame(grid_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.791425,0.064912,3.760904,0.030104,0.0,{'var_smoothing': 1e-09},0.435897,0.301282,0.270299,0.382479,0.361111,0.350214,0.058791,1
1,4.778485,0.022950,3.816845,0.019873,0.0,{'var_smoothing': 1e-08},0.355769,0.193376,0.175214,0.217949,0.287393,0.245940,0.066827,2
2,4.788032,0.014385,3.821704,0.006351,0.0,{'var_smoothing': 1e-07},0.130342,0.117521,0.106838,0.126068,0.134615,0.123077,0.009894,3
3,4.791207,0.028695,3.826829,0.013498,0.000001,{'var_smoothing': 1e-06},0.056624,0.060897,0.048077,0.068376,0.079060,0.062607,0.010524,4
4,4.778437,0.006868,3.825481,0.007353,0.00001,{'var_smoothing': 1e-05},0.028846,0.033120,0.027778,0.041667,0.052350,0.036752,0.009205,5


In [46]:
df[['param_var_smoothing', 'params', 'mean_test_score']]

,param_var_smoothing,params,mean_test_score
0,0.0,{'var_smoothing': 1e-09},0.350214
1,0.0,{'var_smoothing': 1e-08},0.245940
2,0.0,{'var_smoothing': 1e-07},0.123077
3,0.000001,{'var_smoothing': 1e-06},0.062607
4,0.00001,{'var_smoothing': 1e-05},0.036752


In [47]:
grid_search.best_score_

0.3502136752136752

In [48]:
grid_search.best_params_

{'var_smoothing': 1e-09}